## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
from pymystem3 import Mystem
import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('/datasets/data.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [5]:
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


### Вывод

По предварительной оценке данных можно сделать вывод, что столбцы days_employed и total_income имеют пропуски. Стоит отметить, что некоторые значения столбца days_employed отрицательные (количество дней стажа - отрицательное число, теперь понятно почему пенсия такая маленькая у людей), в первых 11 значениях уже обнаружено аномальное значения дней стажа (340266 дней ~ 900 лет стажа, очевидно, что такое развитие событий маловероятно). Беспокоят также и значения в столбце education (то капсом, то нет) - стоит проверить все уникальные значения и привести их к общему формату.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [6]:
# Посмотрим уникальные значения для каждого столбца, особенно нас интересуют children, dob_years, 
# education, family_status, gender, income_type, purpose 
for row in data:
    un_data = data[row].unique()
    print(un_data)
# В столбце children замечено 2 аномальных числа (-1 и 20), 
# в dob_years - одно (0 - только родился, а уже так и тянет взять кредит),
# в gender - одно (XNA), в days_employed - отрицательные и огромные значения стажа.
# На первый взгляд, в остальных столбцах аномалий не обнаружено.

[ 1  0  3  2 -1  4 20  5]
[-8437.67302776 -4024.80375385 -5623.42261023 ... -2113.3468877
 -3112.4817052  -1984.50758853]
[42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51  0 59 29 60 55 58
 71 22 73 66 69 19 72 70 74 75]
['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']
[0 1 2 3 4]
['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']
[0 1 2 3 4]
['F' 'M' 'XNA']
['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']
[0 1]
[253875.6394526  112080.01410244 145885.95229686 ...  89672.56115303
 244093.05050043  82047.41889948]
['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на провед

In [7]:
# Посмотрим количество пропусков (NaN) и заменим их на нули
data.isnull().sum()
data = data.fillna(0)

In [8]:
# Заменим в столбце children аномальные значения
# Есть два подхода замены: первый - предположить, что в данном случае сыграл свою роль человеческий фактор 
# и произошла ошибка, 
# второй - посмотреть медиану значений в столбце и заменить на медиану, так как считаем, что эти значения - заглушка
# Попробуем реализовать второй
median_children = data['children'].median()
data['children'] = data.replace(data.loc[data['children'] == 20], median_children)
data['children'] = data.replace(data.loc[data['children'] == -1], median_children)
data['children'].value_counts()

0.0    14272
1.0     4818
2.0     2055
3.0      330
4.0       41
5.0        9
Name: children, dtype: int64

In [9]:
# Приведем к общему формату значения в столбце education
data['education'] = data['education'].str.lower()
data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [10]:
# Проведем такую же операцию, как и для education, но для family_status.
data['family_status'] = data['family_status'].str.lower()
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [11]:
# Посмотрим какие значения и в каком количестве принимаются в столбце gender 
data['gender'].value_counts()
# Наблюдается аномалия - значение XNA. Не факт, что это выброс. Возможно, что человек - трансгендер.

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [12]:
# Заменим в столбце dob_years число 0 на медиану возраста всех потенциальных клиентов банка
dob_years_median = data['dob_years'].median()
data['dob_years'] = data['dob_years'].replace(0, dob_years_median)
data['dob_years'].value_counts()

42    698
35    617
40    609
41    607
34    603
38    598
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [13]:
# Преобразуем в столбце days_employed отрицательные значения в положительные
data['days_employed'] = data['days_employed'].abs()
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [14]:
# Далее - необходимо избавиться от значений, связанных с аномальным значением стажа
# Для этого определим максимальный возраст людей и будем считать, что трудовую деятельность
# человек начинает с 18 лет
max_days_employed = (data['dob_years'].max() - 18) * 365
max_days_employed

20805

In [15]:
# Медиана возраста = 42 года, 
# для этого возраста найдем медиану дней стажа (mean не подходит, так как есть аномальные значения)
median_days_employed = data.loc[data['dob_years'] == data['dob_years'].median(), 'days_employed'].median()
# Заменим все аномальные значения(те, которые больше максимально возможного стажа) на max_days_employed
data.loc[data['days_employed']>max_days_employed,'days_employed'] = max_days_employed
# Заменим нулевые значения на median_days_employed
data['days_employed'] = data['days_employed'].replace(0, median_days_employed)
# Проверим, произошла ли замена 
data['days_employed'].value_counts()

20805.000000    3445
1843.031892     2174
986.927316         1
5536.801152        1
357.681231         1
                ... 
1216.233315        1
263.131415         1
1218.820922        1
2338.480708        1
582.538413         1
Name: days_employed, Length: 15908, dtype: int64

In [16]:
# Медиана возраста = 42 года, для этого возраста медиану ежемесячного дохода.
data.groupby('dob_years')['total_income'].median()
median_total_income = data.loc[data['dob_years'] == data['dob_years'].median(), 'total_income'].median()
data['total_income'] = data['total_income'].replace(0, median_total_income)
data['total_income'].value_counts()

143494.369652    2174
112874.418757       1
100213.989206       1
122421.963500       1
198271.837248       1
                 ... 
133299.194693       1
115080.782380       1
84896.781597        1
153838.839212       1
150014.128510       1
Name: total_income, Length: 19352, dtype: int64

In [17]:
# Проверим, произошла ли замена всех пропусков и выведем количество уникальных значений всех столбцов
# Мало ли - вдруг произошла неудачная замена и заменились все значения в столбце
data.info()
data.nunique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null object
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(4), object(6)
memory usage: 2.0+ MB


children                6
days_employed       15908
dob_years              57
education               5
education_id            5
family_status           5
family_status_id        5
gender                  3
income_type             8
debt                    2
total_income        19352
purpose                38
dtype: int64

### Вывод


Пропущенные значения были обнаружены типа NaN в столбцах days_employed и total_income. 
Были обнаружены подозрительные значения: 
<ul>
    <li>В столбце children замечено 2 аномальных числа (-1 и 20)</li>
    <li>В dob_years - одно (0)</li>
    <li>В days_employed - отрицательные и огромные значения стажа</li>
</ul>
Причины появления пропусков: 
<ul>
    <li>Человеческий фактор - сокрытие или искажение информации, возникновение случайных ошибок при заполнении</li>
    <li>Технологический фактор - возможна потеря информации при конвертации файла в другой формат</li>
</ul>
В столбцах days_employed и total_income пропуски были заполнены по следующему принципу: 
<ul>
    <li> найдена медиана возраста, исходя из нее - найдена медиана соответствующего значения для данного возраста.  Использование mean в случае days_employed привело бы к аномально большим значениям.</li>
    <li>Для total_income - известно, что для денег использовать среднее = выстрелить себе в ногу, так как очень часто среднее значение не отражает истинную картину</li>
    <li>Для children - произвел замену значений на медиану</li> 
    <li>Для dob_years - произвел замену значения на медиану возраста (можно и на среднее, в данном случае они приблизительно одинаковые)</li>
</ul>   

### Замена типа данных

In [18]:
# Поменяем в столбце days_employed тип данных на int64, чтобы была возможность работать с числовыми значениями 
# И были целочисленные значения, так как нет смысла от дробных значений 
data['days_employed'] = data['days_employed'].astype('int')
# Для столбца children сделаем то же самое
data['children'] = pd.to_numeric(data['children'], errors='coerce')
data['children'] = data['children'].astype('int')
# Приведем значения dob_years и total_income к целочисленным
data['dob_years'] = data['dob_years'].astype('int')
data['total_income'] = data['total_income'].astype('int')

# Проверим
data.info()
data.head()
# Судя по всему - всё в порядке 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,20805,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


### Вывод

Разберем по порядку: 
<ul> 
    <li>В столбце days_employed удобнее пользоваться целочисленными значениями, никому не нужен статус вплоть до 4 знака после запятой</li>
    <li>Видно, что столбец children имеет тип object, что недопустимо, так как все значения там являются числами</li>
    <li>В dob_years и total_income удобнее пользоваться целочисленными значениями, особую роль не сыграют сверхточный возраст(вплоть до дня) и доход до копейки</li>

### Обработка дубликатов

In [19]:
# Посмотрим количество дубликатов
data.duplicated().sum()

72

In [20]:
# Посмотрим размер таблицы
data.shape

(21525, 12)

In [21]:
# Удалим дубликаты 
data = data.drop_duplicates().reset_index(drop=True)

In [22]:
# Посмотрим количество дубликатов после удаления
data.duplicated().sum()

0

In [23]:
# Посмотрим размер таблицы после удаления дубликатов
data.shape

(21453, 12)

### Вывод

В данной таблице легче и рациональнее искать дубликаты методом .duplicated() + .sum(), value_counts() же просто счетчик для уникальных значений. В нашем случае правильным (по моему мнению) методом удаления дубликатов является .drop_duplicates().reset_index(drop=True): .reset_index(drop=True) позволяет удалить дубликаты, не создавая при этом новый столбец с индексацией, при этом индексы записываются без пропусков.

Возможные причины появления дубликатов:
<ul> 
    <li>Ошибки ввода - одно и то же наблюдение было введено несколько раз</li>
    <li>Наличие пропущенных значений</li>
    <li>Изменение формата данных</li>
</ul>

### Лемматизация

In [24]:
# Одна из библиотек с функцией лемматизации на русском языке  - pymystem3, используем модуль Mystem
m = Mystem()

# Так как нам нужно провести лемматизацию для каждой из строк,
# но использование цикла приводит к значительному увеличению времени работы, то используем .apply
data['lemmas_for_purpose'] = data['purpose'].apply(m.lemmatize)
data['lemmas_for_purpose'] = data['lemmas_for_purpose'].apply(''.join)
# Проверим применение лемматизации 
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas_for_purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье\n
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль\n
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье\n
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование\n
4,0,20805,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба\n


In [25]:
# Применим счетчик ради дальнейших выводов
data['lemmas_for_purpose'].value_counts()

автомобиль\n                                972
свадьба\n                                   791
на проведение свадьба\n                     767
сыграть свадьба\n                           765
операция с недвижимость\n                   675
покупка коммерческий недвижимость\n         661
операция с жилье\n                          652
покупка жилье для сдача\n                   651
операция с коммерческий недвижимость\n      650
покупка жилье\n                             646
жилье\n                                     646
покупка жилье для семья\n                   638
строительство собственный недвижимость\n    635
недвижимость\n                              633
операция со свой недвижимость\n             627
строительство жилой недвижимость\n          624
покупка недвижимость\n                      621
покупка свой жилье\n                        620
строительство недвижимость\n                619
ремонт жилье\n                              607
покупка жилой недвижимость\n            

### Вывод

Судя по результату можно сказать, что в колонке purpose много повторяющихся значений, написанных по-разному. В дальнейшем все такие значения будут сведены под каждую из категорий, чтобы было удобнее анализировать и не было потерь данных.

### Категоризация данных

In [26]:
# Категоризация данных нужна непосредственно для Шага 3

# Для зависимости между наличием детей и возвратом кредита в срок
# Необходимые столбцы - children и debt
children_debt_data = data[['children', 'debt']]

# Так как вопрос задан "между наличием детей и...", то необходимо объединить данные столбца children
def children_group(children):
    if children == 0:
        return 'нет детей'
    if children > 0:
        return 'есть дети'
#print(children_group(2))
#print(children_group(0))

# Добавим столбец в children_debt_data 
children_debt_data['children_group'] = children_debt_data['children'].apply(children_group)
children_debt_data.head()

,children,debt,children_group
0,1,0,есть дети
1,1,0,есть дети
2,0,0,нет детей
3,3,0,есть дети
4,0,0,нет детей


In [27]:
# Для зависимости между семейным положением и возвратом кредита в срок
data['family_status'].value_counts()

# Необходимые столбцы - family_status, debt
family_status_debt_data = data[['family_status', 'debt']]
family_status_debt_data.head()

,family_status,debt
0,женат / замужем,0
1,женат / замужем,0
2,женат / замужем,0
3,женат / замужем,0
4,гражданский брак,0


In [28]:
# Для зависимости между уровнем дохода и возвратом кредита в срок
# Необходимые столбцы - total_income и debt
total_income_debt_data = data[['total_income', 'debt']]

# Для удобства анализа необходимо отнести каждое значение ежемесячного дохода в свою категорию
def total_income_group(total_income):
    if total_income < 25000:
        return 'бедные'
    if total_income < 40000:
        return 'выше бедности'
    if total_income < 100000:
        return 'нижний средний класс'
    if total_income < 150000:
        return 'предсредний класс'
    if total_income < 250000:
        return 'cредний класс'
    if total_income < 500000:
        return 'верхний средний класс'
    return 'состоятельные'

#total_income_group(110000)

# Добавим столбец в total_income_debt_data 
total_income_debt_data['total_income_group'] = total_income_debt_data['total_income'].apply(total_income_group)
total_income_debt_data.head()

,total_income,debt,total_income_group
0,253875,0,верхний средний класс
1,112080,0,предсредний класс
2,145885,0,предсредний класс
3,267628,0,верхний средний класс
4,158616,0,cредний класс


In [29]:
# Для зависимости между целями кредита и его возврата в срок
# Необходимые столбцы - purpose и debt
purpose_debt_data = data[['lemmas_for_purpose', 'debt']]


# Так как много одинаковых целей кредита, но написанных по-разному, то создадим функции, которые объединят их в 4 категории
# Были попытки поиска по нужным словам по результатам лемматизации, чтобы оптимизировать код, но они не увенчались успехом,
# Поэтому использовался такой вариант 
def purpose_group(lemmas_for_purpose): 
    if lemmas_for_purpose.find('жилье')!=-1 or lemmas_for_purpose.find('недвижимость')!=-1: 
        return 'недвижимость' 
    if lemmas_for_purpose.find('свадьба')!=-1: 
        return 'свадьба' 
    if lemmas_for_purpose.find('образование')!=-1: 
        return 'образование' 
    if lemmas_for_purpose.find('автомобиль')!=-1: 
        return 'автомобиль' 
# Добавим в отдельный столбец полученный результат
purpose_debt_data['purpose_group'] = purpose_debt_data['lemmas_for_purpose'].apply(purpose_group)
# Проверим этот столбец 
purpose_debt_data['purpose_group'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2323
Name: purpose_group, dtype: int64

### Вывод

Словари были выделены таким образом, чтобы использовать их в Шаге 3.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [30]:
table_child_debt = pd.pivot_table(children_debt_data, index = 'children_group', columns = 'debt', aggfunc = {'debt':'count'})
table_child_debt['procent'] = round((table_child_debt['debt'][1] * 100/ (table_child_debt['debt'][0] + table_child_debt['debt'][1])), 2)
table_child_debt

debt       procent
debt                0     1        
children_group                     
есть дети        6571   669    9.24
нет детей       13141  1072    7.54

### Вывод

Процентное соотношение людей, у которых есть дети и имели проблемы с платежами кредита, больше, чем тех, у кого нет детей и тоже имели проблемы с платежами. Это может быть связано с тем, что появление ребенка в семье приводит к повышенным расходам. Иногда для родителей это может быть неожиданностью, особенно, если ещё "висит" кредит. Поэтому при таком раскладе возможны проблемы с платежами.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?


In [31]:
table_family_debt = pd.pivot_table(family_status_debt_data, index = 'family_status', columns = 'debt', aggfunc = {'debt':'count'})
table_family_debt['percent'] = round((table_family_debt['debt'][1] * 100/ (table_family_debt['debt'][0] + table_family_debt['debt'][1])), 2)
table_family_debt

debt      percent
debt                       0    1        
family_status                            
в разводе               1110   85    7.11
вдовец / вдова           896   63    6.57
гражданский брак        3762  388    9.35
женат / замужем        11408  931    7.55
не женат / не замужем   2536  274    9.75

### Вывод

Самый высокий % проблем с платежами - у неженатых или незамужних. Это может быть связано с тем, что как правило люди менее ответственны, чем другие категории. Дальше следует снижение % в следующем порядке - не женат/не замужем -> гражданский брак -> женат / замужем -> в разводе -> вдовец / вдова. Как ни странно, это похоже на развитие отношений, а следовательно и развития в себе ответственности перед поступками (взятия кредита).

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [32]:
table_income_debt = pd.pivot_table(total_income_debt_data, index = 'total_income_group', columns = 'debt', aggfunc = {'debt':'count'})
table_income_debt['percent'] = round((table_income_debt['debt'][1] * 100/ (table_income_debt['debt'][0] + table_income_debt['debt'][1])), 2)
table_income_debt

debt      percent
debt                      0    1        
total_income_group                      
cредний класс          5840  532    8.35
бедные                    7    1   12.50
верхний средний класс  2410  180    6.95
выше бедности           108    9    7.69
нижний средний класс   3994  344    7.93
предсредний класс      7145  661    8.47
состоятельные           208   14    6.31

### Вывод

Зависимость имеет экстремальный характер: для бедных характерно взять кредит и не возвращать вовремя; дальше идет резкое падение, связанное с повышением дохода классов; на предсреднем классе идет повышение процента - возможно, это связано с более высокой суммой кредита, которая впоследствии дольше выплачивается и выше вероятность иметь проблемы с платежами. В дальнейшем идет спад процента - это связано с повышением доходов (да и заметно, что выборка уменьшается для высоких классов - реже берут кредит, что опять же связано с повышенным доходом).

- Как разные цели кредита влияют на его возврат в срок?

In [33]:
table_debt_purpose = pd.pivot_table(purpose_debt_data, index = 'purpose_group', columns = 'debt', aggfunc = {'debt':'count'})
table_debt_purpose['percent'] = round((table_debt_purpose['debt'][1] * 100/ (table_debt_purpose['debt'][0] + table_debt_purpose['debt'][1])), 2)
table_debt_purpose

debt      percent
debt               0    1        
purpose_group                    
автомобиль      3903  403    9.36
недвижимость   10029  782    7.23
образование     3643  370    9.22
свадьба         2137  186    8.01

### Вывод

Как видно из результатов:
<ul>
    <li> Самый большой процент проблем с платежами имеют кредиты на автомобиль и образование. Вероятно, это связано с тем, что такие кредиты берут людей в молодом возрасте (полно историй формата "учусь в университете, но при этом работаю, чтобы его оплатить")</li>
    <li>Понижение процентов у кредитов на свадьбу связан с тем, что уже некоторое время тенденция повышения возраста, когда люди женяться/выходят замуж (выше возраст, выше вероятность, что человек имеет высокий стабильный доход, значит не должно быть проблем с оплатой долга). Ещё одним из вариантов такого результата может быть то, что на свадьбу большинство людей дарят деньги, которыми можно погасить часть кредита </li>
    <li>Самый низкий процент - у недвижимости. Тут действуют 2 фактора: 
        <ul>
            <li>Осознанность такого выбора, так как сумма кредита гораздо выше, чем у остальных категорий</li>  
            <li>Сложнее всего получить такой кредит</li>
        </ul>
</ul>

### Шаг 4. Общий вывод

Возвращение кредита в срок не зависит только от одного фактора: 
<ul>
    <li> Наличие детей увеличивает долю людей, имеющих проблемы с кредитами</li>
    <li> Большой опыт отношений и семейной жизни увеличивает вероятность, что кредит будет оплачен в срок</li>
    <li> Уровень дохода имеет сложную зависимость, но общая тенденция такая - бедным сложнее выплатить кредит, чем среднему классу, состоятельным людям же легче</li>
    <li> Цель кредита имеет немаловажную роль - кредит на авто и образование берут люди в молодом возрасте, на свадьбе - в более осознанном возрасте, так как есть тенденция жениться/выходить замуж в более зрелом возрасте, на недвижимость - сложность взятия кредита</li>
</ul>
Таким образом, чтобы создать хорошую модель для прогнозирования возвращения кредита, необходимо учитывать несколько факторов.

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.